# Stock Price(QQQ) Prediction Project

QQQ is an ETF tracking the NASDAQ-100 Index, bundling major U.S. tech stocks like Apple and Tesla.  
With over 50% weighting in the tech sector, it directly mirrors the growth trajectory of technology industries.  
While highly volatile due to its tech concentration, it has historically delivered strong long-term returns (~15% annual average).   
Short-term performance remains sensitive to interest rate changes and macroeconomic conditions.  

For our price prediction project, we'll focus on analyzing tech trends, earnings reports, and Fed policies as key variables.

## 1. Libraries & APIs

https://www.kaggle.com/code/carlmcbrideellis/lstm-time-series-stock-price-prediction-fail

In [ ]:
python --version

In [ ]:
%pip install tensorflow

In [43]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from matplotlib.pylab import rcParams
import seaborn as sns
from numpy.random import seed
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

ImportError: Traceback (most recent call last):
  File "C:\Users\bpark\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
## Stock dataset install

!pip -q install yfinance
import yfinance as yf

In [ ]:
pip show yfinance

## 2. Load Data

In this project, the goal is short-term prediction, and to avoid overfitting, it was determined that using 5 years of data is appropriate.  
Therefore, the period was set from 2020 to 2024.

In [ ]:
# download using yhfinance
df= yf.download("QQQ", start="2020-01-01", end="2024-12-31", auto_adjust=False)

In [ ]:
# reset_index to add the data column
df = df.reset_index()
df.head()

In [ ]:
# Rename columns
df.columns = ['date', 'adj close', 'close', 'high', 'low', 'open', 'volume']
df.head()

In [ ]:
# delete later
#df.to_csv('QQQ adj_close (200101-241231).csv', index=False)

In [ ]:
df.info()

In [ ]:
print("Null Values:", df.isnull().values.sum())
print("NA Values:", df.isna().values.any())

## 3. EDA(Exploratory Data Analysis)

### 3.1. Duration of Dataset

In [ ]:
print("Starting date: ", df.iat[0, 0])
print("Ending date: ", df.iat[-1, 0])
print("Total number of days: ",df.shape[0])
print("Total number of fields: ",df.shape[1])

### 3.2. Stock Price Trends

First, it is necessary to visualize the stock price trends from 2020 to 2024 to intuitively understand the data.  
This will help quickly identify trends, volatility, outliers, and specific patterns, providing valuable insights for model building.

In [ ]:
# Visualize 5 years of stock price data
fig, ax = plt.subplots(figsize=(10, 4))

ax.plot(df['date'], df['close'])
ax.set_xlabel('Years')
ax.set_ylabel('Closing Price')
fig.autofmt_xdate()

plt.tight_layout()
plt.show()

Through the above graph, we can understand the overall trend over the past 5 years. Overall, there is an upward trend, but there are sharp declines in the early part of 2020 and between 2022 and 2023. To examine this in more detail, let's look at the graphs for each year.

In [ ]:
# Visualizing 5 years of data using a 3x2 subplot layout
fig, axes = plt.subplots(3, 2, figsize=(10, 8), sharex=False, sharey=True)

years = [2020, 2021, 2022, 2023, 2024]

# Plot each year's data
for i, year in enumerate(years):
    row, col = i // 2, i % 2 
    df_year = df[df['date'].dt.year == year]
    
    # Plot only if data is available
    if not df_year.empty:
        axes[row, col].plot(df_year['date'], df_year['close'])
        axes[row, col].set_title(f"Stock Prices in {year}")
        axes[row, col].set_ylabel("Closing Price")
        
        # Set X-axis limits from the start to the end of the year
        axes[row, col].set_xlim(df_year['date'].min(), df_year['date'].max())

# Remove empty subplots if any
for i in range(len(years), 6):  
    fig.delaxes(axes[i // 2, i % 2])

# Automatically adjust X-axis date format
fig.autofmt_xdate()

# Adjust layout and display the plots
plt.tight_layout()
plt.show()


### 📈 Key Events Impacting the Stock Market in 2020, 2022

- **2020.02**  
**COVID-19 pandemic** broke out, increasing global uncertainty and causing a sharp decline in stock prices.
Later, a rebound occurred as government stimulus measures and central bank interest rate cuts helped the stock market recover.

- **2022.02**   
**Russia vs. Ukraine war** began.
Energy and commodity prices soared, leading to worsening global economic inflation.
Increased uncertainty caused stock prices to drop.

- **2022.03**   
**Fed's interest rate hike (0.25%) began**, aimed at controlling inflation, but it negatively impacted the stock market.

- **2022.06, 07, 09**   
**Fed's interest rate hikes (each 0.75%)** were implemented.
The June and July rate hikes especially caused significant market volatility, further deepening the downward trend in the stock market.

### 3.3. Seasonality, Yearly Trend

To understand the seasonality, trends, and patterns in the stock market over the past five years, we calculate the monthly and yearly average closing prices.

**Seasonality Check**  
The stock market can exhibit recurring patterns at specific times of the year. For example, phenomena like the Year-end Rally or low trading volumes during the summer are commonly observed in the market. By calculating the average closing price per month, we can identify which months tend to show bullish or bearish trends.

**Yearly Trend Analysis**  
To understand the long-term market trends, we conduct a yearly average closing price analysis. This allows us to assess the overall market flow in each year and identify periods of significant volatility.

In [ ]:
# Extract the month and year from the 'Date' column
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year


# Calculate the mean close price per month
monthly_mean = df.groupby(['month'])['close'].mean().reset_index()

# Calculate the mean close price per year
yearly_mean = df.groupby(['year'])['close'].mean().reset_index()

print("\n<Monthly Mean Close Price> \n", monthly_mean)
print("\n<Yearly Mean Close Price> \n", yearly_mean)

#### 3.3.1 Seasonality Check

In [ ]:
# Plot the mean close price per month using a Seaborn bar chart
ax = sns.barplot(data=monthly_mean, x='month', y='close', hue='month', palette='coolwarm', legend=False)

plt.title('Mean Close Price per Month')
plt.xlabel('Month')
plt.ylabel('Mean Close Price')

# X-axis ticks for each month
plt.xticks(range(0, 12), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                  'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# Adding the value labels on top of bars
for bars in ax.containers:
    ax.bar_label(bars, fontsize=8, fmt='%.2f')

plt.show()

### 📈 **Monthly Seasonality (Strong Bullish/Bearish Periods)**

**Bullish in the Second Half** (July to December)  
From June (338) to December (378), a +11.8% increase.  
Especially a sharp rise from November (369) to December (378) → aligns with the "Year-end Rally" phenomenon.

**Volatility in the First Half** (January to March)  
From January (320) to March (316), a -1.3% decrease → no sign of the "January Effect" (traditionally a bullish January).

**Summer Stagnation** (July to September)  
From July (355) to September (354), a slight decline of -0.3%.

#### 3.3.2 Yearly Trend Analysis

In [ ]:
lst_year= yearly_mean['year'].tolist()

# Plot the mean close price per month using a Seaborn bar chart
ax = sns.barplot(data=yearly_mean, x='year', y='close', palette='coolwarm', hue = 'year', legend = False)

plt.title('Mean Close Price per Year')
plt.xlabel('Year')
plt.ylabel('Mean Close Price')

plt.xticks(range(0, len(lst_year)), labels=lst_year)

for bars in ax.containers:
     ax.bar_label(bars, fontsize=12, fmt='%.2f')

plt.show()

### 📊 **Annual Trend (5-Year Long-Term Performance)**

Annual Average Return from 2020 to 2024: +16.7% (250 → 464)

**2021:** +40.9% (250 → 353) → Tech sector boom

**2022:** -12.0% (353 → 310) → Impact of interest rate hikes

**2024:** +34.4% (345 → 464) → AI rally driving growth

## 4. Feature Engineering

- Moving Averages (MA): Smooths stock price movements to identify short-term and long-term trends (e.g., 5-day, 20-day, 50-day, 200-day).
- Relative Strength Index (RSI): Measures whether a stock is overbought or oversold, reflecting price momentum.
- MACD (Moving Average Convergence Divergence): Identifies trend changes and momentum using two moving averages.
- Bollinger Bands: Measures price volatility and deviation from the average price.
- Volume Volatility: Analyzes the relationship between price and trading volume to predict market movements.
- Open-Close Difference: Indicates daily price volatility and potential trend strength.
- High-Low Difference: Represents intraday volatility, with larger differences signaling higher fluctuations.
- High-Open Difference (High_Open_Diff): Measures how much the high price (high) has increased compared to the opening price (open) during the day.
- Low-Close Difference (Low_Close_Diff): Measures how much the closing price (close) has recovered compared to the low price (low) during the day.

In [ ]:
# 1. Moving Average (MA)
df['MA_5'] = df['close'].rolling(window=5).mean()  # 5-day moving average
df['MA_20'] = df['close'].rolling(window=20).mean()  # 20-day moving average
df['MA_50'] = df['close'].rolling(window=50).mean()  # 50-day moving average

# 2. Relative Strength Index (RSI)
delta = df['close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

# 3. MACD (Moving Average Convergence Divergence)
df['EMA_12'] = df['close'].ewm(span=12, adjust=False).mean()  # 12-day exponential moving average
df['EMA_26'] = df['close'].ewm(span=26, adjust=False).mean()  # 26-day exponential moving average
df['MACD'] = df['EMA_12'] - df['EMA_26']

# 4. Bollinger Bands
df['Bollinger_Upper'] = df['MA_20'] + 2 * df['close'].rolling(window=20).std() 
df['Bollinger_Lower'] = df['MA_20'] - 2 * df['close'].rolling(window=20).std()

# 5. Volume
df['Volume_MA_20'] = df['volume'].rolling(window=20).mean()  # 20-day moving average of volume

# 6. Open-Close, High-Low Difference
df['Open_Close_Diff'] = df['open'] - df['close'] 
df['High_Low_Diff'] = df['high'] - df['low'] 

# 8. High-Open Difference, Low-Close Difference
df['High_Open_Diff'] = df['high'] - df['open'] 
df['Low_Close_Diff'] = df['low'] - df['close'] 

In [ ]:
df.info()

## Modeling

### 1. Baseline Model for benchmark

In [ ]:
### Moving Average Model : Linear

In [ ]:
# Calculate 5-day Moving Average (MA_5)
df['MA_5'] = df['close'].rolling(window=5).mean()

# Use the 5-day Moving Average as the baseline model's prediction
# Since we want to predict the next day's closing price, shift the MA_5 by 1 day
df['predicted_close_ma'] = df['MA_5'].shift(1)

# Remove the NaN values caused by the shift
df = df.dropna(subset=['predicted_close_ma'])

# Define the features and target for the MA model
X_baseline = df[['MA_5']]  # Using the MA_5 as the feature
y_baseline = df['close']   # Actual close prices are the target

# Train-test split
X_train_baseline, X_test_baseline, y_train_baseline, y_test_baseline = train_test_split(X_baseline, y_baseline, test_size=0.2, random_state=42)

# Train a model (Linear Regression)
from sklearn.linear_model import LinearRegression

baseline_model = LinearRegression()
baseline_model.fit(X_train_baseline, y_train_baseline)

# Prediction
y_pred_baseline = baseline_model.predict(X_test_baseline)

# Evaluate the baseline model
mae_baseline = mean_absolute_error(y_test_baseline, y_pred_baseline)
print(f"Baseline MAE (Moving Average): {mae_baseline}")

### 2. XGBoost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

# Fill missing values
df.ffill(inplace=True)

# Define the features and target variable
# Use the technical indicators (excluding date and adj close)
features = ['MA_5', 'MA_20', 'MA_50', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'Bollinger_Upper', 'Bollinger_Lower', 
            'Volume_MA_20', 'Open_Close_Diff', 'High_Low_Diff', 'High_Open_Diff', 'Low_Close_Diff']
target = 'close'

X = df[features]
y = df[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model (with tuned hyperparameters)
model = xgb.XGBRegressor(
    n_estimators=1000, #EX
    learning_rate=0.01, #EX
    max_depth=6, #EX
    subsample=0.8,#EX
    colsample_bytree=0.8, #EX
    random_state=42
)

model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# plot feature importance
xgb.plot_importance(model, importance_type='weight')
plt.show()

In [ ]:
# 1. Feature Scaling (LSTM에 입력되는 값들은 0~1로 정규화)
scaler = MinMaxScaler(feature_range=(0, 1))

# 'close'만 정규화 (타겟만 사용할 예정)
scaled_data = scaler.fit_transform(df['close'].values.reshape(-1, 1))

# 2. 시계열 데이터 생성 (Window size - 예: 60일)
def create_dataset(data, window_size=60):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i, 0])  # X는 윈도우 크기만큼의 데이터
        y.append(data[i, 0])  # y는 다음 날의 종가
    return np.array(X), np.array(y)

window_size = 60  # 60일 데이터를 기반으로 예측
X, y = create_dataset(scaled_data, window_size)

# 3. 데이터 형태 조정 (LSTM의 입력 형태에 맞게)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # (samples, timesteps, features)

# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. LSTM 모델 구성
model = Sequential()

# LSTM layer
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# LSTM layer
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Fully connected layer
model.add(Dense(units=1))

# 6. 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 7. 모델 훈련
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# 8. 예측
y_pred = model.predict(X_test)

# 9. 결과 시각화
plt.figure(figsize=(12, 6))
plt.plot(y_test, color='blue', label='Actual Stock Price')
plt.plot(y_pred, color='red', label='Predicted Stock Price')
plt.title('Stock Price Prediction using LSTM')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

# 10. 평가 (MAE 계산)
from sklearn.metrics import mean_absolute_error
mae_lstm = mean_absolute_error(y_test, y_pred)
print(f"LSTM MAE: {mae_lstm}")